# Parse PDF File for table

In [ ]:
# Install necessary libraries 
!pip install pdfplumber

In [ ]:
import pdfplumber

pdf_path = "..\..\private\client-files\collin.pdf"  # Insert the path to the PDF file
csv_file_path = "..\..\private\client-files\collin_college_csv" # Insert the path to the CSV file

with pdfplumber.open(pdf_path) as pdf:
    extracted_text = ""
    tables = []

    for page_num, page in enumerate(pdf.pages):
        extracted_text += page.extract_text() + "\n\n"
        tables_on_page = page.extract_tables()

        if tables_on_page:
            for table in tables_on_page:
                tables.append(table)

In [ ]:
for i, table in enumerate(tables):
    print(f"\nTable {i + 1}:")
    for row in table:
        print(row)

In [ ]:
all_rows = []

for table in tables:
    for row in table:
        all_rows.append(row)

split_tables = []
current_table = []


for row in all_rows:
    if len(row) > 2 and row[2] == "Phone":
        if current_table:
            split_tables.append(current_table)
        current_table = [row]
    else:
        current_table.append(row)

if current_table:
    split_tables.append(current_table)

In [ ]:
for i, split_table in enumerate(split_tables):
    print(f"\nSplit Table {i + 1}:")
    for row in split_table:
        print(row)

In [ ]:
filtered_tables = []
extraneous_tables = []
for i, split_table in enumerate(split_tables):
    filtered_table = []
    extraneous_table = []
    for row in split_table:
        if any(cell not in [None, ""] for cell in row):
            if any(cell in [None, ""] for cell in row) or len(row) < 3:
                extraneous_table.append(row.copy())
            else:
                filtered_table.append(row.copy())

    print(f"\nSplit Table {i + 1} (after removing empty rows):")
    for row in filtered_table:
        print(row)
    print(f"\nExtra Table {i + 1} (need to augment the split table):")
    for row in extraneous_table:
        print(row)

    filtered_tables.append(filtered_table)
    extraneous_tables.append(extraneous_table)

In [ ]:
# Augmenting the extra to the split table
index = next(i for i, row in enumerate(filtered_tables[4]) if "National Autism" in row[0])
print(filtered_tables[4][index][1], "\n")
filtered_tables[4][index][1] += "\n" + [" ".join(row[1] for row in extraneous_tables[4])][0]
print(filtered_tables[4][index][1], "\n")

index = next(i for i, row in enumerate(filtered_tables[5]) if "Life Path" in row[0])
print(filtered_tables[5][index][1], "\n")
filtered_tables[5][index][1] += " " + [" ".join(extraneous_tables[5][i][0] for i in range(0, 4))][0]
print(filtered_tables[5][index][1], "\n")


filtered_tables[8].append(extraneous_tables[8][0])
print(filtered_tables[8], "\n")

index = next(i for i, row in enumerate(filtered_tables[9]) if "Medical City" in row[0])
print(filtered_tables[9][index][0], "\n")
filtered_tables[9][index][0] += "\n" + [" ".join(extraneous_tables[9][i][0] for i in range(0, 2))][0]
filtered_tables[9][index][0] += "\n" + extraneous_tables[9][2][0].replace("\n", "")
print(filtered_tables[9][index][0], "\n")

index = next(i for i, row in enumerate(filtered_tables[12]) if "Collin County Health Care Services" in row[0])
print(filtered_tables[12][index][0], "\n")
filtered_tables[12][index][0] += extraneous_tables[12][0][0]
print(filtered_tables[12][index][0], "\n")

index = next(i for i, row in enumerate(filtered_tables[12]) if "Women’s Veteran" in row[0])
print(filtered_tables[12][index][0], "\n")
filtered_tables[12][index][0] += extraneous_tables[12][1][0]
print(filtered_tables[12][index][0], "\n")

index = next(i for i, row in enumerate(filtered_tables[13]) if "Islamic" in row[0])
print(filtered_tables[13][index][1], "\n")
filtered_tables[13][index][1] += "\n" + ["\n".join(extraneous_tables[13][i][1] for i in range(0, 4))][0]
print(filtered_tables[13][index][1], "\n")

filtered_tables[13].append(extraneous_tables[13][4])
print(filtered_tables[13], "\n")

In [ ]:
filtered_table = [[
    [cell.replace('\n', ' ') if idx == 1 else cell for idx, cell in enumerate(row)] 
    for row in filtered_table
] for filtered_table in filtered_tables]

for i, test_table in enumerate(filtered_table):
    print(f"\nTest Table {i + 1}:")
    for row in test_table:
        print(row[1]) if len(row) > 1 else print(row)

In [173]:
for i, test_table in enumerate(filtered_table):
    for row in test_table:
        lines = [line.strip() for line in row[0].split("\n")]
        # The /A is outlier in the data with care/Adoption mess up
        possible_links = [line for line in lines if "/" in line and '/A' not in line]
        possible_links = [line.replace(" ", "") for line in possible_links]